# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), November 20, 2020**

In [1]:
using DataFrames

In [2]:
using BenchmarkTools

In [3]:
using CategoricalArrays

In [4]:
using PooledArrays

## Performance tips

### Access by column number is faster than by name

In [5]:
x = DataFrame(rand(5, 1000), :auto)
@btime $x[!, 500];
@btime $x.x500;

  5.099 ns (0 allocations: 0 bytes)
  13.239 ns (0 allocations: 0 bytes)


### When working with data `DataFrame` use barrier functions or type annotation

In [6]:
using Random
function f_bad() # this function will be slow
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    y, z = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_bad();
# if you run @code_warntype f_bad() then you notice
# that Julia does not know column types of `DataFrame`


  110.717 ms (5999011 allocations: 122.06 MiB)


In [7]:
# solution 1 is to use barrier function (it should be possible to use it in almost any code)
function f_inner(y,z)
   p = 0.0
   for i in 1:length(y)
       p += y[i]*z[i]
   end
   p
end

function f_barrier() # extract the work to an inner function
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    f_inner(x[!, 1], x[!, 2])
end

using LinearAlgebra
function f_inbuilt() # or use inbuilt function if possible
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    dot(x[!, 1], x[!, 2])
end

@btime f_barrier();
@btime f_inbuilt();

  9.821 ms (33 allocations: 30.52 MiB)
  9.323 ms (33 allocations: 30.52 MiB)


In [8]:
# solution 2 is to provide the types of extracted columns
# it is simpler but there are cases in which you will not know these types
# This example  assumes that you have DataFrames master at least from August 31, 2018
function f_typed()
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    y::Vector{Float64}, z::Vector{Float64} = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_typed();

  9.723 ms (33 allocations: 30.52 MiB)


In general for tall and narrow tables it is often useful to use `Tables.rowtable`, `Tables.columntable` or `Tables.namedtupleiterator` for intermediate processing of data in a type-stable way.

### Consider using delayed `DataFrame` creation technique

also notice the difference in performance between copying vs non-copying data frame creation

In [9]:
function f1()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100], :auto, copycols=false) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f1a()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100], :auto) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f2()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x, :auto, copycols=false) # we delay creation of DataFrame after we have our job done
end

function f2a()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x, :auto) # we delay creation of DataFrame after we have our job done
end

@btime f1();
@btime f1a();
@btime f2();
@btime f2a();

  26.882 ms (1949725 allocations: 37.42 MiB)
  29.538 ms (1949925 allocations: 45.05 MiB)
  3.740 ms (825 allocations: 7.68 MiB)
  4.905 ms (1025 allocations: 15.32 MiB)


### You can add rows to a `DataFrame` in place and it is fast

In [10]:
x = DataFrame(rand(10^6, 5), :auto)
y = DataFrame(transpose(1.0:5.0), :auto)
z = [1.0:5.0;]

@btime vcat($x, $y); # creates a new DataFrame - slow
@btime append!($x, $y); # in place - fast

x = DataFrame(rand(10^6, 5), :auto) # reset to the same starting point
@btime push!($x, $z); # add a single row in place - fast

  10.179 ms (186 allocations: 38.16 MiB)
  1.100 μs (17 allocations: 832 bytes)
  498.964 ns (16 allocations: 256 bytes)


### Allowing `missing` as well as `categorical` slows down computations

In [11]:
using StatsBase

function test(data) # uses countmap function to test performance
    println(eltype(data))
    x = rand(data, 10^6)
    y = categorical(x)
    println(" raw:")
    @btime countmap($x)
    println(" categorical:")
    @btime countmap($y)
    nothing
end

test(1:10)
test([randstring() for i in 1:10])
test(allowmissing(1:10))
test(allowmissing([randstring() for i in 1:10]))


Int64
 raw:
  4.178 ms (7 allocations: 7.63 MiB)
 categorical:
  16.534 ms (4 allocations: 608 bytes)
String
 raw:
  26.217 ms (4 allocations: 608 bytes)
 categorical:
  27.760 ms (4 allocations: 608 bytes)
Union{Missing, Int64}
 raw:
  9.386 ms (4 allocations: 624 bytes)
 categorical:
  17.038 ms (4 allocations: 608 bytes)
Union{Missing, String}
 raw:
  42.756 ms (4 allocations: 608 bytes)
 categorical:
  38.164 ms (4 allocations: 608 bytes)


### When aggregating use column selector and prefer categorical or pooled array grouping variable

In [12]:
df = DataFrame(x=rand('a':'d', 10^7), y=1);

In [13]:
gdf = groupby(df, :x)

,x,y
,Char,Int64
1,b,1
2,b,1
3,b,1
4,b,1
5,b,1
6,b,1
7,b,1
8,b,1
9,b,1


In [14]:
@btime combine(v -> sum(v.y), $gdf) # traditional syntax, slow

  50.879 ms (217 allocations: 76.31 MiB)


,x,x1
,Char,Int64
1,b,2498877
2,a,2499759
3,c,2499778
4,d,2501586


In [15]:
@btime combine($gdf, :y=>sum) # use column selector

  9.445 ms (151 allocations: 11.92 KiB)


,x,y_sum
,Char,Int64
1,b,2498877
2,a,2499759
3,c,2499778
4,d,2501586


In [16]:
transform!(df, :x => categorical => :x);

In [17]:
gdf = groupby(df, :x)

,x,y
,Cat…,Int64
1,a,1
2,a,1
3,a,1
4,a,1
5,a,1
6,a,1
7,a,1
8,a,1
9,a,1


In [18]:
@btime combine($gdf, :y=>sum)

  9.252 ms (163 allocations: 12.80 KiB)


,x,y_sum
,Cat…,Int64
1,a,2499759
2,b,2498877
3,c,2499778
4,d,2501586


In [19]:
transform!(df, :x => PooledArray{Char} => :x)

,x,y
,Char,Int64
1,b,1
2,b,1
3,a,1
4,b,1
5,a,1
6,a,1
7,c,1
8,a,1
9,d,1


In [20]:
gdf = groupby(df, :x)

,x,y
,Char,Int64
1,b,1
2,b,1
3,b,1
4,b,1
5,b,1
6,b,1
7,b,1
8,b,1
9,b,1


In [21]:
@btime combine($gdf, :y=>sum)

  9.344 ms (158 allocations: 12.50 KiB)


,x,y_sum
,Char,Int64
1,b,2498877
2,a,2499759
3,c,2499778
4,d,2501586


### Use views instead of materializing a new DataFrame

In [22]:
x = DataFrame(rand(100, 1000), :auto)

,x1,x2,x3,x4,x5,x6,x7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.654616,0.719149,0.152413,0.149021,0.906955,0.601046,0.659532
2,0.200366,0.77317,0.54823,0.619689,0.311176,0.348622,0.116158
3,0.0707588,0.282814,0.809332,0.553294,0.6751,0.187457,0.640539
4,0.263277,0.974213,0.469272,0.976958,0.862772,0.938512,0.617304
5,0.58163,0.949869,0.0661904,0.863933,0.685727,0.395609,0.311059
6,0.616957,0.809582,0.667691,0.0241465,0.437346,0.833641,0.11238
7,0.321904,0.569951,0.581407,0.520908,0.34152,0.570203,0.269444
8,0.862673,0.992583,0.153743,0.641501,0.479036,0.858642,0.0335735
9,0.974797,0.185455,0.183458,0.793523,0.525708,0.263889,0.572278


In [23]:
@btime $x[1:1, :]

  166.701 μs (1986 allocations: 193.33 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.654616,0.719149,0.152413,0.149021,0.906955,0.601046,0.659532,0.49023,0.414895


In [24]:
@btime $x[1, :]

  24.823 ns (0 allocations: 0 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.654616,0.719149,0.152413,0.149021,0.906955,0.601046,0.659532,0.49023,0.414895


In [25]:
@btime view($x, 1:1, :)

  24.823 ns (0 allocations: 0 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.654616,0.719149,0.152413,0.149021,0.906955,0.601046,0.659532,0.49023,0.414895


In [26]:
@btime $x[1:1, 1:20]

  5.071 μs (39 allocations: 6.17 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.654616,0.719149,0.152413,0.149021,0.906955,0.601046,0.659532,0.49023,0.414895


In [27]:
@btime $x[1, 1:20]

  27.939 ns (0 allocations: 0 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.654616,0.719149,0.152413,0.149021,0.906955,0.601046,0.659532,0.49023,0.414895


In [28]:
@btime view($x, 1:1, 1:20)

  26.532 ns (0 allocations: 0 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.654616,0.719149,0.152413,0.149021,0.906955,0.601046,0.659532,0.49023,0.414895
